## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-05-50-58 +0000,nypost,Notorious Ecuadorian drug kingpin Wilmer ‘Pipo...,https://nypost.com/2025/11/17/world-news/ecuad...
1,2025-11-17-05-44-34 +0000,bbc,Trump calls on House Republicans to vote to re...,https://www.bbc.com/news/articles/cvgv653v1vjo...
2,2025-11-17-05-38-04 +0000,nypost,Germany cancels Holocaust artifacts auction af...,https://nypost.com/2025/11/17/world-news/germa...
3,2025-11-17-05-36-32 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...
4,2025-11-17-05-25-33 +0000,nyt,"At a Shared Symbol of Jihad, the Taliban Defy ...",https://www.nytimes.com/2025/11/17/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,27
16,epstein,11
59,new,10
296,will,8
17,files,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...,82
28,2025-11-17-02-44-24 +0000,nypost,Trump says House Republicans should vote to re...,https://nypost.com/2025/11/16/us-news/trump-sa...,67
138,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,65
3,2025-11-17-05-36-32 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...,63
91,2025-11-16-21-32-28 +0000,bbc,Marjorie Taylor Greene doubles down on Epstein...,https://www.bbc.com/news/articles/cdeg8z0j2j2o...,61


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,82,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...
95,39,2025-11-16-21-02-08 +0000,latimes,Body of 7-year-old girl is found off Big Sur; ...,https://www.latimes.com/california/story/2025-...
58,33,2025-11-17-00-19-40 +0000,nypost,"Liberal pols, DSA warn Gov. Kathy Hochul to ba...",https://nypost.com/2025/11/16/us-news/liberals...
151,32,2025-11-16-13-28-00 +0000,wsj,New Rare-Earths Plant in Europe Shows How Toug...,https://www.wsj.com/world/europe/a-new-rare-ea...
73,28,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
111,26,2025-11-16-19-47-46 +0000,cbc,U.S. stations its most advanced aircraft carri...,https://www.cbc.ca/news/world/us-venezuela-air...
76,24,2025-11-16-23-08-20 +0000,nypost,YouTube star Jack Doherty challenged Miami cop...,https://nypost.com/2025/11/16/us-news/youtube-...
42,22,2025-11-17-01-38-39 +0000,nypost,FAA lifts order slashing flights — here’s when...,https://nypost.com/2025/11/16/us-news/faa-lift...
137,22,2025-11-16-17-00-00 +0000,wsj,Over 60% of Americans are comfortable with how...,https://www.wsj.com/politics/policy/economic-o...
70,21,2025-11-16-23-42-32 +0000,nypost,Bearded cane-toting man shoots guy on NYC stre...,https://nypost.com/2025/11/16/us-news/bearded-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
